In [1]:
from evotorch import Problem
from evotorch.algorithms import GeneticAlgorithm
from evotorch.operators import OnePointCrossOver, GaussianMutation, Operator
from torch import Tensor
import random
import torch
import math
import logging
log = logging.getLogger("evotorch")
log.propagate = False
log.setLevel(logging.ERROR)

In [2]:
def distance(x, y):
    return math.sqrt((x[0]-y[0])**2+(x[1]-y[1])**2)


def fun(x):
    #Huge penalty for non unique indexes
    uniq = x.unique(sorted=False)
    if(uniq.size(0) != x.size(0)):
        return (x.size(0)-uniq.size(0))*penalty
    dist = distance(cityList[x[x.size(0) - 1]], cityList[x[0]])
    for i in range(x.size(0) - 1):
        dist += distance(cityList[x[i]], cityList[x[i + 1]])
    return dist

def mutation(values: torch.Tensor) -> torch.Tensor:
    for j in range(values.size(0)):
        u = values[j].unique(sorted=False)
        #Makes indexes all unique
        if(u.size(0) != city_count):
            all = list(range(city_count))
            for i in u:
                all.remove(i)
            random.shuffle(all)
            values[j] = torch.cat([u, torch.Tensor(all)])
            
        #Swaps values in population randomly
        for i in range(swap_count):
            r1 = random.randint(0, city_count-1)
            r2 = random.randint(0, city_count-2)
            if(r2 >= r1):
                r2 += 1
            values[j][r1], values[j][r2] = values[j][r2], values[j][r1]
    return values


def create_city_list(city_count: int) -> list:
    val_range = city_count*10
    return [
        (random.randrange(val_range), random.randrange(val_range))
        for _ in range(city_count)
    ]

In [3]:
city_count = 100
max_dist = 0
cityList = create_city_list(city_count)

for i in cityList:
    for j in cityList:
        dist = distance(i, j)
        if(dist > max_dist):
            max_dist = dist
penalty = max_dist * len(cityList)

In [4]:
import networkx as nx
G = nx.cycle_graph(city_count)
for i in range(city_count):
    for j in range(city_count):
        G.add_edge(i, j, weight=distance(cityList[i], cityList[j]))
path = nx.approximation.traveling_salesman_problem(G)
dist = 0
for i in range(len(path)-1):
    dist += distance(cityList[path[i]], cityList[path[i+1]])

In [5]:
min = 999999999999

In [6]:
popsize = math.floor(city_count*0.01)
tournament_size = popsize

In [ ]:
for swap_count in range(math.floor(city_count*0.01), popsize, math.floor(city_count*0.01)):

        problem = Problem("min", fun, bounds=(0, city_count-1), solution_length=city_count, dtype=torch.int64)
        searcher = GeneticAlgorithm(problem, popsize=popsize, operators=[
            OnePointCrossOver(problem, tournament_size=tournament_size),
            mutation
        ])
        searcher.run(500)
        ans = searcher.status["best_eval"]
        print(f"popsize: {popsize} swap_count: {swap_count} ans: {ans}")
        if(ans < min):
            min = ans
            print(f"popsize: {popsize} swap_count: {swap_count}")
            print(f"perfect: {dist}")
            print(path[:-1])
            print()
            print(f"ai: {ans}")
            print(searcher.status["best"].values.tolist())
            print()
            print(f"diff: {ans-dist}")
            print(f"magnitude: {ans/dist}")
popsize += math.floor(city_count*0.01)

popsize: 7 swap_count: 1 ans: 130675.5546875
popsize: 7 swap_count: 2 ans: 130675.5546875
popsize: 7 swap_count: 3 ans: 130675.5546875
popsize: 7 swap_count: 4 ans: 261351.109375
popsize: 7 swap_count: 5 ans: 392026.65625
